In [13]:
from notebookutils import mssparkutils

def Bronze_ReadData():
    paras = {'inferschema' : True , 'header' : True} 
    path = "Files/FinData_Bronze" 
    files = mssparkutils.fs.ls(path)
    df_list = {}

    for file in files:
        # print(f"{file.path}")
        df = spark.read.format('csv').options(**paras).load(file.path)
        # display(df)
        name = file.name.split(".")[0]
        df_list[name] = df
    return df_list
    

StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 15, Finished, Available, Finished)

In [14]:
def validate_bank_data(df):
    from pyspark.sql.functions import col

    df = df.dropDuplicates()
    df = df.dropna(how='all') 
    df = df.fillna({'Firm_Revenue': 0}) 
    df = df.withColumn('Firm_Revenue' , col('Firm_Revenue').cast('int')) 
    df = df.orderBy('Branch_ID')
    return df

StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 16, Finished, Available, Finished)

In [15]:
def validate_customer_data(df): 
    from pyspark.sql.functions import col

    df = df.dropDuplicates()
    df = df.dropna(how='all') 
    df = df.fillna({'Age' : 0 , 'Customer_Type' : 'Unknown' , 'City' : 'Unknown'})
    df = df.withColumn('Age' , col('Age').cast('int'))
    df = df.orderBy('Customer_ID')
    return df

StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 17, Finished, Available, Finished)

In [16]:
def validate_transaction_data(df):
    from pyspark.sql.functions import col ,to_date

    df = df.dropDuplicates()
    df = df.dropna(how='all') 
    df = df.withColumn('Transaction_Amount' , col('Transaction_Amount').cast('int'))
    df = df.withColumn('Transaction_Date' , to_date(col('Transaction_Date'), 'yyyy-MM-dd'))
    df = df.orderBy('Transaction_ID')
    return df

StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 18, Finished, Available, Finished)

In [17]:
dfs = Bronze_ReadData()

bank_df_cleaned = validate_bank_data(dfs['bank_data'])
customer_df_cleaned = validate_customer_data(dfs['customer_data'])
transaction_df_cleaned = validate_transaction_data(dfs['transaction_data'])

display(bank_df_cleaned)
display(customer_df_cleaned)
display(transaction_df_cleaned)


StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b1dd71cd-e492-4e65-82e4-c9262e7d456a)

SynapseWidget(Synapse.DataFrame, ee450f10-2307-41ca-8316-7f7976b5803f)

SynapseWidget(Synapse.DataFrame, 8aad9e54-118e-46b7-aae2-d7b95568a5e9)

In [18]:
bank_df_cleaned.write.mode("append").format("delta").save("Files/FinData_Silver/Silver_bank_data")
customer_df_cleaned.write.mode("append").format("delta").save("Files/FinData_Silver/Silver_customer_data")
transaction_df_cleaned.write.mode("append").format("delta").save("Files/FinData_Silver/Silver_transaction_data")


StatementMeta(, a63a8c6e-c9fc-4d48-be2b-979cde0d7ecf, 20, Finished, Available, Finished)